In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import paired_distances,cosine_similarity

In [2]:
jobs = pd.read_csv('/Users/weberwen/Documents/資料競賽檔案/jobs.csv')
rate = pd.read_csv('/Users/weberwen/Documents/資料競賽檔案/ratings_job.csv')

display(jobs.head())
display(rate.head())

,OCCU_DESC,CJOB_TYPE,CJOB_NAME1,CJOB_NO,CJOB_NAME2
0,海外財務專員,4,財務／金融／保險,40109,財務專業人員
1,【寶雅】 門市服務人員((台北台視店),6,客服／門市,60202,賣場（人員／儲備幹部）
2,客服專員,6,客服／門市,60102,客戶服務人員
3,兼職動物照護助理(兼職),22,其他職類,220101,兼職人員
4,業務助理,2,業務／貿易／銷售,20103,業務助理


,userId,CJOB_TYPE,rating,timestamp
0,1,4.0,4.0,964982703
1,1,6.0,4.0,964981247
2,1,6.0,4.0,964982224
3,1,22.0,5.0,964983815
4,1,2.0,5.0,964982931


In [3]:
# jobs留下CJOB_TYPE與CJOB_NAME1(職缺分類)
# rate留下userId與CJOB_TYPE
# 把jobs與rate以userId合併成一個df

jobs.drop(['OCCU_DESC','CJOB_NO','CJOB_NAME2'],axis=1,inplace=True)
rate.drop(['rating', 'timestamp'],axis=1,inplace=True)
df = pd.merge(rate, jobs, on='CJOB_TYPE')
df.head()

,userId,CJOB_TYPE,CJOB_NAME1
0,1,4.0,財務／金融／保險
1,1,4.0,財務／金融／保險
2,1,4.0,財務／金融／保險
3,1,4.0,財務／金融／保險
4,1,4.0,財務／金融／保險


In [4]:
# 建立job的特徵矩陣
oneHot = jobs["CJOB_NAME1"].str.get_dummies("|") # One-Hot Encoding
job_arr = pd.concat([jobs, oneHot], axis=1)
job_arr.drop("CJOB_NAME1",axis=1,inplace=True)
job_arr.set_index("CJOB_TYPE",inplace=True)
display(job_arr.head())

# 建立user的特徵矩陣
oneHot = df["CJOB_NAME1"].str.get_dummies("|") # One-Hot Encoding
user_arr = pd.concat([df, oneHot], axis=1)
user_arr.drop(["CJOB_TYPE","CJOB_NAME1"],axis=1,inplace=True)
user_arr = user_arr.groupby('userId').mean()
display(user_arr.head())

,人資／法務／智財,保全／軍警消,傳播／娛樂／藝術,其他職類,品管／製造／環衛,客服／門市,工程／研發／生技,廣告／公關／設計,技術／維修／操作,教育／學術／研究,...,業務／貿易／銷售,清潔／家事／托育(保母),營建／製圖／施作,物流／運輸／資材,經營／行政／總務,行銷／企劃／專案,財務／金融／保險,資訊／軟體／系統,農林漁牧相關,醫療／美容／保健
CJOB_TYPE,,,,,,,,,,,,,,,,,,,,,
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
6,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


,人資／法務／智財,保全／軍警消,傳播／娛樂／藝術,其他職類,品管／製造／環衛,客服／門市,工程／研發／生技,廣告／公關／設計,技術／維修／操作,教育／學術／研究,...,業務／貿易／銷售,清潔／家事／托育(保母),營建／製圖／施作,物流／運輸／資材,經營／行政／總務,行銷／企劃／專案,財務／金融／保險,資訊／軟體／系統,農林漁牧相關,醫療／美容／保健
userId,,,,,,,,,,,,,,,,,,,,,
1,0.001923,0.015016,0.0,0.001099,0.006592,0.074986,0.010712,0.003113,0.021974,0.007142,...,0.053058,0.071690,0.012818,0.020646,0.065922,0.010987,0.011719,0.022661,0.000000,0.021242
2,0.000000,0.035043,0.0,0.000000,0.000000,0.100000,0.000000,0.021795,0.000000,0.011111,...,0.026068,0.074359,0.034188,0.035043,0.061538,0.012821,0.041026,0.038462,0.000000,0.000000
3,0.003696,0.032471,0.0,0.000000,0.000000,0.041183,0.006864,0.000000,0.012672,0.000000,...,0.016103,0.160771,0.021119,0.021647,0.019007,0.007920,0.008448,0.035639,0.001056,0.045935
4,0.003927,0.017249,0.0,0.002244,0.008975,0.061983,0.012154,0.002384,0.024681,0.006077,...,0.039920,0.048801,0.009349,0.009583,0.043753,0.005609,0.008975,0.027345,0.000187,0.032534
5,0.000000,0.044907,0.0,0.000000,0.006572,0.106791,0.007119,0.000000,0.013143,0.028478,...,0.016703,0.047645,0.054765,0.022453,0.059146,0.016429,0.026287,0.012322,0.000000,0.015882


In [5]:
# user-job相似度矩陣
similar_matrix = cosine_similarity(user_arr.values, job_arr.values)
similar_matrix = pd.DataFrame(similar_matrix, index = user_arr.index, columns = job_arr.index)
similar_matrix

CJOB_TYPE,4,6,6,22,2,7,19,1,10,10,...,6,1,14,16,17,16,16,10,19,11
userId,,,,,,,,,,,,,,,,,,,,,
1,0.020050,0.128287,0.128287,0.001880,0.090772,0.018327,0.122648,0.112780,0.037593,0.037593,...,0.128287,0.112780,0.012218,0.969281,0.036340,0.969281,0.969281,0.037593,0.122648,0.021929
2,0.076691,0.186935,0.186935,0.000000,0.048731,0.000000,0.139003,0.115037,0.000000,0.000000,...,0.186935,0.115037,0.020771,0.950653,0.000000,0.950653,0.950653,0.000000,0.139003,0.063909
3,0.014206,0.069254,0.069254,0.000000,0.027080,0.011542,0.270356,0.031963,0.021309,0.021309,...,0.069254,0.031963,0.000000,0.950906,0.077244,0.950906,0.950906,0.021309,0.270356,0.035515
4,0.013929,0.096199,0.096199,0.003482,0.061956,0.018862,0.075740,0.067905,0.038305,0.038305,...,0.096199,0.067905,0.009431,0.984186,0.050493,0.984186,0.984186,0.038305,0.075740,0.014510
5,0.048216,0.195879,0.195879,0.000000,0.030638,0.013059,0.087392,0.108487,0.024108,0.024108,...,0.195879,0.108487,0.052234,0.956293,0.029131,0.956293,0.956293,0.024108,0.087392,0.100451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,0.000000,0.151886,0.151886,0.000000,0.059391,0.000000,0.254117,0.070101,0.046734,0.046734,...,0.151886,0.070101,0.050629,0.926894,0.169411,0.926894,0.926894,0.046734,0.254117,0.038945
61,0.031909,0.038889,0.038889,0.005983,0.060826,0.012963,0.260255,0.107692,0.047863,0.047863,...,0.038889,0.107692,0.000000,0.949282,0.057834,0.949282,0.949282,0.047863,0.260255,0.039886
62,0.011190,0.088649,0.088649,0.003147,0.058662,0.005683,0.121696,0.088124,0.048259,0.048259,...,0.088649,0.088124,0.009092,0.977936,0.065918,0.977936,0.977936,0.048259,0.121696,0.027976


In [6]:
# 取得與特定user最相似的前num部job
def get_the_most_similar_jobs(searchUserId, num):
  vec = similar_matrix.loc[searchUserId].values
  sorted_index = np.argsort(-vec)[:num]   #找距離最短
  return list(similar_matrix.columns[sorted_index])

# 取得與特定job最相似的前num部job
def get_the_most_similar_users(searchjobId, num):
  job_vec = similar_matrix[searchjobId].values 
  sorted_index = np.argsort(-job_vec)[:num]  #找距離最短
  return list(similar_matrix.index[sorted_index])  

In [23]:
# sort最相似的資料
searchjobId = 1
searchUserId = 12
num = 5

In [24]:
similar_jobs_index = get_the_most_similar_jobs(searchUserId, num)
print(similar_jobs_index)

[16, 16, 16, 16, 16]


In [25]:
# 重新讀入jobs.csv為了要有title
jobs = pd.read_csv('/Users/weberwen/Documents/資料競賽檔案/jobs.csv')

# 列出推薦名單
df_recommend_jobs = pd.DataFrame({f'推薦給[使用者類型{searchUserId}]的前{num}個推薦職缺類型':jobs[jobs.CJOB_TYPE.isin(similar_jobs_index)].CJOB_NAME1[:num]}).reset_index()
df_recommend_jobs.drop('index',axis=1,inplace=True)

pd.concat([df_recommend_jobs],axis=1)

,推薦給[使用者類型12]的前5個推薦職缺類型
0,旅遊／餐飲／休閒
1,旅遊／餐飲／休閒
2,旅遊／餐飲／休閒
3,旅遊／餐飲／休閒
4,旅遊／餐飲／休閒


## 總結來說，第二種類型的人格 最適合的前五類職缺都是旅遊/餐飲這類的，我們會優先推薦她/他這類型的職缺